In [1]:
import numpy as np; np.random.seed(1218042)
import seaborn as sb; sb.set()
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm_notebook as tqdm 

# Population Model of Competition between ride hailing platforms

Let $f_{R}(t, u)$ and $f_{D}(t, u)$ model the local densities of riders and drivers. Then their growths are determined by:

$$
\frac{\delta}{\delta t}f_{R}(t, u) = R_{R}(t, u)f_{R}(t, u)
$$
$$
\frac{\delta}{\delta t}f_{D}(t, v) = R_{D}(t, v)f_{D}(t, v)
$$

where $R_{R}$ and $R_{D}$ model the net growth rates of rider and drivers respectively:

$$
R_{R}(t, u) := (\kappa_{R} - \mu_{R}\rho_{R}(t)) - \int_{V} \eta_{\theta_{P}}(|u-v|)f_{D}(t, v)dv
$$
$$
R_{D}(t, v) := [\int_{U}\eta_{\theta_{P}}(|u-v|)f_{R}(t, u)du + \kappa_{D}c_{I}(t)] - \frac{\mu_{D}}{1+\mu_{A}c_{A}(t)}\rho_{D}(t)
$$

The implementation is as follows:

#### For riders:

* $\kappa_{R}$ the natural growth rate of rider agents,
* $\mu_{R}$ the reduction in platform attractiveness due to competition for ressources among riders (**waiting time effect**),
* $\rho_{R}(t)$ the total density of riders that are currently seeking rides (**unmet demant**),
* $\eta_{\theta_{P}}$ the price surge in effect with $\theta_{P}$ being the surge coefficient,
* $f_{D}(t, v)$ the density of drivers for platform $v$,
* $u$ indicator of riders for platform $u$,
* $V$ the set of platforms that drivers can join.


#### For drivers:

* $\eta_{\theta_{P}}$ the price surge in effect with $\theta_{P}$ being the surge coefficient,
* $f_{R}(t, u)$ the density of riders for platform $u$,
* $\kappa_{D}$ the natural growth rate of driver agents (**barabasi effect on growth**),
* $c_{I}(t)$ the effect of incentives presented to drivers for them to join platform $v$ (*e.g. financial, advertisment, ...*),
* $\mu_{D}$ the reduction of platform attractiveness for drivers due to competition for ressources amongst drivers (**idle time**),
* $\mu_{A}$ the reduction of platform attractiveness for drivers due to driver attrition,
* $c_{A}(t)$ the effect of external mitigations on driver attrition (*e.g. good employee development programs*),
* $\rho_{D}(t)$ the total density of drivers that are currently seeking rides,
* $v$ indicator of drivers for platform $u$,
* $U$ the set of platforms that riders can join.

## Modelling the growth

To recreate the model, $f_{R,I}$ will be an array of its values where each index reprents its value at time $t$.

At every time step ($\Delta t = 0.1$):

$$f_{R}(t_{k+1}, u) = f_{R}(t_{k}, u) + \Delta t [R^{+}_{R}(t_{k}, u)f_{R}(t_{k}, u) - R^{-}_{R}(t_{k}, u)f_{R}(t_{k+1}, u)]$$
$$f_{D}(t_{k+1}, v) = f_{D}(t_{k}, v) + \Delta t [R^{+}_{D}(t_{k}, v)f_{D}(t_{k}, v) - R^{-}_{D}(t_{k}, v)f_{D}(t_{k+1}, v)]$$

or in explicit terms:

$$f_{R}(t_{k+1}, u) = f_{R}(t_{k}, u) \frac{1+\Delta t R^{+}_{R}(t_{k}, u)}{1+\Delta t R^{-}_{R}(t_{k}, u)} \qquad f_{D}(t_{k+1}, v) = f_{D}(t_{k}, v) \frac{1+\Delta t R^{+}_{D}(t_{k}, v)}{1+\Delta t R^{-}_{D}(t_{k}, v)}$$

We will compute this evolution for various sets of platforms $U$ and $V$.